In [25]:
# download tiny Shakespears dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [26]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f'length of dataset = {len(text)}')

length of dataset = 1115393


In [27]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [28]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [29]:
# build a simple encoder
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # input: string, output: list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # input: list of integers, output: string
print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [30]:
import torch
print(torch.__version__)

2.0.1+cu117


In [31]:
# encode the whole dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [32]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [33]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [34]:
x = train_data[: block_size]
y = train_data[1: block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when context is {context}, target is {target}')

when context is tensor([18]), target is 47
when context is tensor([18, 47]), target is 56
when context is tensor([18, 47, 56]), target is 57
when context is tensor([18, 47, 56, 57]), target is 58
when context is tensor([18, 47, 56, 57, 58]), target is 1
when context is tensor([18, 47, 56, 57, 58,  1]), target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [35]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # randomly select the starting index of text blocks
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when context is {context.tolist()}, target is {target}")

when context is [53], target is 59
when context is [53, 59], target is 6
when context is [53, 59, 6], target is 1
when context is [53, 59, 6, 1], target is 58
when context is [53, 59, 6, 1, 58], target is 56
when context is [53, 59, 6, 1, 58, 56], target is 47
when context is [53, 59, 6, 1, 58, 56, 47], target is 40
when context is [53, 59, 6, 1, 58, 56, 47, 40], target is 59
when context is [49], target is 43
when context is [49, 43], target is 43
when context is [49, 43, 43], target is 54
when context is [49, 43, 43, 54], target is 1
when context is [49, 43, 43, 54, 1], target is 47
when context is [49, 43, 43, 54, 1, 47], target is 58
when context is [49, 43, 43, 54, 1, 47, 58], target is 1
when context is [49, 43, 43, 54, 1, 47, 58, 1], target is 58
when context is [13], target is 52
when context is [13, 52], target is 45
when context is [13, 52, 45], target is 43
when context is [13, 52, 45, 43], target is 50
when context is [13, 52, 45, 43, 50], target is 53
when context is [13, 

In [44]:
# implement a bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embeding_table(idx) # shape: (B, T, C) = (batch, time, channel) = (batch_size, block_size, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1) # B *T 
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        '''
        Takes in an input of shape (B,T), generate output of shape (B, T + max_new_tokens) 
        '''
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            # convert logits to probabilities
            prob = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(prob, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

m = BigramLanguageModel(vocab_size=vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [49]:
# let's train the model
optimizer = torch.optim.AdamW(m.parameters(), lr=1.e-3)
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.367445707321167


In [50]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



WI anod I avin,
Hor.
I nind
OUKENNGr ly thound wow mpiterounts; Moth ird t ld hy wilo ithy couthe c


In [59]:
B, T, C = 4, 8, 2
x = torch.rand(B, T, C)
x_bow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1, :]
        x_bow[b, t] = torch.mean(xprev, dim=0)

wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
x_bow2 = wei @ x
print(torch.allclose(x_bow, x_bow2))

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
x_bow3 = wei @ x
print(torch.allclose(x_bow, x_bow3))

True
True


In [57]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
print(a)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
